In [4]:
from datetime import datetime, timedelta
import time
import requests

# Polygon API 
API_KEY = "beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq"
BASE_URL = "https://api.polygon.io/v2/aggs/ticker"

# fetch polygon data
def fetch_forex_rate(pair):
    url = f"{BASE_URL}/{pair}/prev?unadjusted=true&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['results'][0]['c'] if data['results'] else None
    else:
        print("Error fetching data:", response.text)
        return None

# track pair and print start time
def track_forex_pairs(interval='hour', test_duration=3):
    initial_investment = 100
    pairs = {"C:GBPUSD": "long", "C:USDJPY": "short"}
    rates = {}
    quantities = {}
    ratios = []
    returns = []
    
    start_time = datetime.now()
    end_time = start_time + timedelta(hours=test_duration)

    print(f"Start time: {start_time}")

    # set inital ratio and quantity
    for pair in pairs:
        rate = fetch_forex_rate(pair)
        if rate:
            if pair == "C:USDJPY":
                rate /= 100
            rates[pair] = rate

    base_unit = initial_investment / rates["C:GBPUSD"]
    quantities["C:GBPUSD"] = base_unit
    quantities["C:USDJPY"] = base_unit * rates["C:GBPUSD"] / rates["C:USDJPY"]

    # calculate ratio
    last_ratio = quantities["C:GBPUSD"] * rates["C:GBPUSD"] / (quantities["C:USDJPY"] * rates["C:USDJPY"])
    ratios.append(last_ratio)

    # for loop, update rate
    for hour in range(1, test_duration + 1):
        time.sleep(3600)  # wait for next hour

        for pair in pairs:
            rate = fetch_forex_rate(pair)
            if rate:
                if pair == "C:USDJPY":
                    rate /= 100
                rates[pair] = rate

        current_ratio = quantities["C:GBPUSD"] * rates["C:GBPUSD"] / (quantities["C:USDJPY"] * rates["C:USDJPY"])
        ratios.append(current_ratio)
        current_return = (current_ratio / last_ratio - 1) * 100
        returns.append(current_return)
        last_ratio = current_ratio

        print(f"\nCurrent Time: {datetime.now()}")
        print(f"GBPUSD/USDJPY ratio at {datetime.now()}: {current_ratio:.6f}")
        print(f"Return this {interval}: {current_return:.6f}%")

    return ratios, returns

# 运行函数并打印结果
ratios, returns = track_forex_pairs(test_duration=3) 
print("Final Ratios:", [f"{ratio:.6f}" for ratio in ratios])
print("Hourly Returns:", [f"{ret:.6f}" for ret in returns])

Start time: 2024-05-03 00:01:27.034740

Current Time: 2024-05-03 01:01:27.276229
GBPUSD/USDJPY ratio at 2024-05-03 01:01:27.276229: 1.000000
Return this hour: 0.000000%

Current Time: 2024-05-03 02:01:27.437265
GBPUSD/USDJPY ratio at 2024-05-03 02:01:27.438275: 1.000000
Return this hour: 0.000000%

Current Time: 2024-05-03 03:01:24.473519
GBPUSD/USDJPY ratio at 2024-05-03 03:01:24.473519: 0.999203
Return this hour: -0.079667%
Final Ratios: ['1.000000', '1.000000', '1.000000', '0.999203']
Hourly Returns: ['0.000000', '0.000000', '-0.079667']
